# Digital Finance Programming Final Project
##  20204654, 우지환
### The risk position for delta bucket b, $K_b$

---
### 0. 주요 패키지를 import 한다. 
### 1. 파일 'equity.xlsx', 'table.xlsx'를 읽는다.

In [1]:
import pandas as pd
import numpy as np
import copy

equity_file_name = 'equity.xlsx'
table_file_name = 'tables.xlsx'

equity_data = pd.read_excel(equity_file_name)
table_data  = pd.read_excel(table_file_name, sheet_name= 'table10', header=1)

### 2. Data 내용 확인 
#### 2.1 Equity 내용 확인

In [2]:
# Head 내용 확인
equity_data.head()

,risk measure,risk class,risk factor type,bucket,issuer name,stock name,net sensitivity
0,Delta,Equity,equity spot price,1,A,A,100
1,Delta,Equity,equity spot price,1,B,B,100
2,Delta,Equity,equity spot price,1,C,C,100
3,Delta,Equity,equity spot price,2,A,A,100
4,Delta,Equity,equity spot price,2,B,B,100


In [3]:
# Tail 내용 확인
equity_data.tail()

,risk measure,risk class,risk factor type,bucket,issuer name,stock name,net sensitivity
60,Delta,Equity,equity repo rate,11,B,B,100
61,Delta,Equity,equity repo rate,12,A,A,100
62,Delta,Equity,equity repo rate,12,B,B,100
63,Delta,Equity,equity repo rate,13,A,A,100
64,Delta,Equity,equity repo rate,13,B,B,100


#### 2.2 Table 내용 확인

In [4]:
# Head 내용 확인
table_data.head()

,Bucket number,Risk weight for equity spot price,Risk weight for equity repo rate,Correlation
0,1,0.55,0.0055,0.15
1,2,0.60,0.0060,0.15
2,3,0.45,0.0045,0.15
3,4,0.55,0.0055,0.15
4,5,0.30,0.0030,0.25


In [5]:
# Tail 내용 확인
table_data.tail()

,Bucket number,Risk weight for equity spot price,Risk weight for equity repo rate,Correlation
8,9,0.70,0.0070,0.075
9,10,0.50,0.0050,0.125
10,11,0.70,0.0070,NaN
11,12,0.15,0.0015,0.800
12,13,0.25,0.0025,0.800


### 3. $WS_k$ 계산
### $WS_k = RW_k * s_k$
### $RW_k$: Risk weight for equity spot price or Risk Weight for equity repo rate (Risk Factor Type에 따라 결정)
### $s_k$: Net Sensitivity, equity_data의 7번째 열
### The weighted sensitivity $WS_k$is the product of the net sensitivity $s_k$ and the corresponding risk weight $RW_k$ as defined in [MAR21.39] to [MAR21.95].

-----
### 3.1. Net Sensitivity 데이터 입력 및 확인

In [6]:
# Define s_k: 0 <= k < num_equity_data
num_equity_data = np.size(equity_data,0)
Sensitivity = np.zeros(shape=(num_equity_data,1))
Sensitivity = equity_data['net sensitivity'].to_numpy()

# Display Sensitivity Vector
print('Sensitivity, shortly s_k,', 'Vector size=', Sensitivity.shape)
print(Sensitivity)

Sensitivity, shortly s_k, Vector size= (65,)
[100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100]


-----
### 3.2. Risk Weight 데이터 입력 및 확인

In [7]:
# Define RW_k: 0<= k <num_equity_data
Risk_Weight = np.zeros(shape=(num_equity_data,1))
iter = 0
for buck_num,type in zip(equity_data['bucket'], equity_data['risk factor type']):
    if type == 'equity spot price':
        Risk_Weight[iter]= table_data['Risk weight for equity spot price'].to_numpy()[buck_num-1]
    else:
        Risk_Weight[iter]= table_data['Risk weight for equity repo rate'].to_numpy()[buck_num-1]
    iter = iter + 1
Risk_Weight= np.squeeze(Risk_Weight, axis=1)

# Display Risk Weight Vector
print('Risk_Weight, RW_k', 'Vector Size=', Risk_Weight.shape)
print(Risk_Weight)

Risk_Weight, RW_k Vector Size= (65,)
[0.55   0.55   0.55   0.6    0.6    0.6    0.45   0.45   0.45   0.55
 0.55   0.55   0.3    0.3    0.3    0.35   0.35   0.35   0.4    0.4
 0.4    0.5    0.5    0.5    0.7    0.7    0.7    0.5    0.5    0.5
 0.7    0.7    0.7    0.15   0.15   0.15   0.25   0.25   0.25   0.0055
 0.0055 0.006  0.006  0.0045 0.0045 0.0055 0.0055 0.003  0.003  0.0035
 0.0035 0.004  0.004  0.005  0.005  0.007  0.007  0.005  0.005  0.007
 0.007  0.0015 0.0015 0.0025 0.0025]


-----
### 3.3. Weighted Sensitivity 계산
### $WS_k = RW_k * s_k$

In [8]:
# Define WS_k: 0<= k <num_equity_data
Weight_Sensitivity = Risk_Weight * Sensitivity

## Disply Weighted_Sensitivity Vector
print('Weighted Sensitivity Vector,' 'Vector Size=', Weight_Sensitivity.shape)
print(Weight_Sensitivity)

Weighted Sensitivity Vector,Vector Size= (65,)
[55.   55.   55.   60.   60.   60.   45.   45.   45.   55.   55.   55.
 30.   30.   30.   35.   35.   35.   40.   40.   40.   50.   50.   50.
 70.   70.   70.   50.   50.   50.   70.   70.   70.   15.   15.   15.
 25.   25.   25.    0.55  0.55  0.6   0.6   0.45  0.45  0.55  0.55  0.3
  0.3   0.35  0.35  0.4   0.4   0.5   0.5   0.7   0.7   0.5   0.5   0.7
  0.7   0.15  0.15  0.25  0.25]


-----
### 4. Correlation parameter $\rho_{𝑘l}$ between two sensitivities $𝑊S_𝑘$ and $𝑊S_l$
$$K_b=\sqrt {max(0, \sum_{k}WS_{k}^2 + \sum_{k}\sum_{k\neq l} \rho_{kl}WS_{k}WS_{l})}$$

__21.78 For aggregating delta equity risk positions within a bucket, the correlation parameter $\rho_{𝑘l}$ between two sensitivities $𝑊S_𝑘$ and $𝑊S_l$ within the same bucket is set at as follows__ 

_(1) The correlation parameter $\rho_{𝑘l}$ is set at 99.90%, where:_  
(a) one is a sensitivity to an equity spot price and the other a sensitivity to an equity repo rates; and  
(b) both are related to the same equity issuer name.   
(2) The correlation parameter $𝜌_{𝑘l}$ is set out in     
(a) to (d) below, where both sensitivities are to equity spot price, and where:_
(a) 15% between two sensitivities within the same bucket that fall under large market cap, emerging market economy (bucket number 1, 2, 3 or 4).  
(b) 25% between two sensitivities within the same bucket that fall under large market cap, advanced economy (bucket number 5, 6, 7 or 8).  
(c) 7.5% between two sensitivities within the same bucket that fall under small market cap, emerging market economy (bucket number 9).  
(d) 12.5% between two sensitivities within the same bucket that fall under small market cap, advanced economy (bucket number 10).  
(e) 80% between two sensitivities within the same bucket that fall under either index bucket (bucket number 12 or 13)  
_(3) The same correlation parameter $\rho_{𝑘l}$ as set out in above (2)(a) to (d) apply, where both sensitivities are to equity repo rates._  
_(4) The correlation parameter $\rho_{𝑘l}$ is set as each parameter specified in above (2)(a) to (d) multiplied by 99.90%, where:_  
(a) One is a sensitivity to an equity spot price and the other a sensitivity to an equity repo rate; and  
(b) Each sensitivity is related to a different equity issuer name.  

__21.79 The correlations set out above do not apply to the other sector bucket (ie bucket 11).  
(1) The aggregation of equity risk positions within the other sector bucket capital requirement would be equal to the simple sum of the absolute values of the net weighted sensitivities allocated to this bucket. The same method applies to the aggregation of vega risk positions.__

__21.6 In order to address the risk that correlations increase or decrease in periods of financial stress, the aggregation of bucket level capital requirements and risk class level capital requirements per each risk class for delta, vega, and curvature risks as specified in [MAR21.4] to [MAR21.5] must be repeated, corresponding to three different scenarios on the specified values for the correlation parameter $\rho_{𝑘l}$ (correlation between risk factors within a bucket) and $\gamma_{𝑘l}$  (correlation across buckets within a risk class).__  
(1) Under the “medium correlations” scenario, the correlation parameters $\rho_{𝑘l}$ as specified in [MAR21.39] to [MAR21.101] apply.  
(2) Under the “high correlations” scenario, the correlation parameters $\rho_{𝑘l}$ that is specified in [MAR21.39] to [MAR21.101] are uniformly multiplied by 1.25, with $\rho_{𝑘l}$ subject to a cap at 100%.  
(3) Under the “low correlations” scenario, the correlation parameters $\rho_{𝑘l}$ that is specified in MAR21.39 to MAR21.101] are replaced by $\rho_{𝑘l} = max(2 × \rho_{kl} − 1, 0.75 × \rho_{kl})$.

In [9]:
# Rho를 계산하기 위한 데이터 확인 작업: 
# 같은 Bucket에 속한 Equity에 대해서 Risk Factor Type과, Issuer Name, Weighted Sensitivity를 계산
Bucket = list(range(13))
for id in Bucket:
    With_in_Bucket = equity_data[equity_data['bucket']==id+1]
    With_in_Bucket_index = np.where(equity_data['bucket'].to_numpy() == id+1)
    
    # Loading Weighted Sensitivity, Risk Factor Type, and Issuer Name for calculating correlation, rho_kl
    Risk_Factor_Type = With_in_Bucket['risk factor type'].to_numpy()
    Issuer_Name = With_in_Bucket['issuer name'].to_numpy()
    WS = Weight_Sensitivity[With_in_Bucket_index]
    
    print('bucket:', id+1)
    print('1) Equity ID:', With_in_Bucket_index)
    print('2) Risk Factor Type:', Risk_Factor_Type)
    print('3) Issuer Name:', Issuer_Name)
    print('4) Weighted Sensitivity:', WS)
    print('--------------------------------------------------------')

bucket: 1
1) Equity ID: (array([ 0,  1,  2, 39, 40]),)
2) Risk Factor Type: ['equity spot price' 'equity spot price' 'equity spot price'
 'equity repo rate' 'equity repo rate']
3) Issuer Name: ['A' 'B' 'C' 'A' 'B']
4) Weighted Sensitivity: [55.   55.   55.    0.55  0.55]
--------------------------------------------------------
bucket: 2
1) Equity ID: (array([ 3,  4,  5, 41, 42]),)
2) Risk Factor Type: ['equity spot price' 'equity spot price' 'equity spot price'
 'equity repo rate' 'equity repo rate']
3) Issuer Name: ['A' 'B' 'C' 'A' 'B']
4) Weighted Sensitivity: [60.  60.  60.   0.6  0.6]
--------------------------------------------------------
bucket: 3
1) Equity ID: (array([ 6,  7,  8, 43, 44]),)
2) Risk Factor Type: ['equity spot price' 'equity spot price' 'equity spot price'
 'equity repo rate' 'equity repo rate']
3) Issuer Name: ['A' 'B' 'C' 'A' 'B']
4) Weighted Sensitivity: [45.   45.   45.    0.45  0.45]
--------------------------------------------------------
bucket: 4
1) Equit

In [10]:
# Array for saving Delta Risk Table 
K_b = np.zeros((13,3))

# Display the Array
print(K_b)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


### 21.78, 21.79 Test: Bucket 1에 대해서 Test
### k, l 두 equity에 대해서, Risk Factor Type과 Issuer Name을 비교해서 $\rho_{kl}$을 계산함

In [11]:
With_in_Bucket = equity_data[equity_data['bucket']==1]
# the number of equity in the Bucket id+1
len = np.size(With_in_Bucket,0)
# First Summation (Sigma for all k)
for k in range(len):
    # Second Summation(Sigma for all l != k)
    print('%d-th: RFType = %s, Issuer = %s' % (k+1, Risk_Factor_Type[k],Issuer_Name[k]))
    for l in range(len):
        if k==l:
            continue
        else:
            if Risk_Factor_Type[k] != Risk_Factor_Type[l]:
                if Issuer_Name[k] == Issuer_Name[l]:
                    # According to 21.78 (1)
                    Rho_kl = 0.9990             
                else:
                    # 21.78 (4)
                    Rho_kl = 0.9990 * table_data['Correlation'][id]
            else:
                #  According to 21.78 (2), (3)
                Rho_kl = table_data['Correlation'][id] 
                
            #  According to 21.78 (2), (3)
            Rho_kl_medium = Rho_kl
            # According to 21.6 (2)
            Rho_kl_high = min(Rho_kl * 1.25, 1)
            # According to 21.6 (3)
            Rho_kl_low = max(2 * Rho_kl - 1, 0.75 * Rho_kl)
            print('(%d,%d)' % (k+1,l+1), '|RFType = %s, Issuer = %s' % (Risk_Factor_Type[l],Issuer_Name[l]), 
                  '|Rho(medium) = %3.3f, Rho(high) = %3.3f, Rho(low) = %3.3f' % (Rho_kl_medium, Rho_kl_high,Rho_kl_low))
    print('----------------------------------------------------------------------------------------------------------')

1-th: RFType = equity spot price, Issuer = A
(1,2) |RFType = equity spot price, Issuer = B |Rho(medium) = 0.800, Rho(high) = 1.000, Rho(low) = 0.600
(1,3) |RFType = equity spot price, Issuer = C |Rho(medium) = 0.800, Rho(high) = 1.000, Rho(low) = 0.600
(1,4) |RFType = equity repo rate, Issuer = A |Rho(medium) = 0.999, Rho(high) = 1.000, Rho(low) = 0.998
(1,5) |RFType = equity repo rate, Issuer = B |Rho(medium) = 0.799, Rho(high) = 0.999, Rho(low) = 0.599
----------------------------------------------------------------------------------------------------------
2-th: RFType = equity spot price, Issuer = B
(2,1) |RFType = equity spot price, Issuer = A |Rho(medium) = 0.800, Rho(high) = 1.000, Rho(low) = 0.600
(2,3) |RFType = equity spot price, Issuer = C |Rho(medium) = 0.800, Rho(high) = 1.000, Rho(low) = 0.600
(2,4) |RFType = equity repo rate, Issuer = A |Rho(medium) = 0.799, Rho(high) = 0.999, Rho(low) = 0.599
(2,5) |RFType = equity repo rate, Issuer = B |Rho(medium) = 0.999, Rho(high) =

### Delta Risk Table 만드는 과정

In [12]:
Bucket = list(range(13))
# Varaiable for saving Delta Risk Table 
K_b = np.zeros((13,3))

# Processing for all Bucket
for id in Bucket:
    # Searching for Equity index which is belonging to same Bucket
    With_in_Bucket = equity_data[equity_data['bucket']==id+1]
    With_in_Bucket_index = np.where(equity_data['bucket'].to_numpy() == id+1)
    
    # Loading Weighted Sensitivity, Risk Factor Type, and Issuer Name for calculating correlation, rho_kl
    Risk_Factor_Type = With_in_Bucket['risk factor type'].to_numpy()
    Issuer_Name = With_in_Bucket['issuer name'].to_numpy()
    WS = Weight_Sensitivity[With_in_Bucket_index]
    
    # the number of equity in the Bucket id+1
    len = np.size(With_in_Bucket,0)
    
    # Other sector bucket (Only for Bucket 11)
    if id == 10:
        for k in range(len):
            # According to 21.79 (1)
            K_b[id] += np.abs(WS[k])
    
    # within a bucket (Bucket 1,2,3,4,5,6,7,8,9,10,12,13)
    else:    
        Sigma_WS_k_square = 0
        Sigma_Sigma_Rho_WS_k_WS_l_medium = 0
        Sigma_Sigma_Rho_WS_k_WS_l_high = 0
        Sigma_Sigma_Rho_WS_k_WS_l_low = 0

        # First Summation (Sigma for k)
        for k in range(len):
            # Second Summation(Sigma for all l != k)
            for l in range(len):
                if k==l:
                    Sigma_WS_k_square += WS[k] * WS[l]
                else:
                    if Risk_Factor_Type[k] != Risk_Factor_Type[l]:
                        if Issuer_Name[k] == Issuer_Name[l]:
                            # According to 21.78 (1)
                            Rho_kl = 0.9990             
                        else:
                            # 21.78 (4)
                            Rho_kl = 0.9990 * table_data['Correlation'][id]
                    else:
                        #  According to 21.78 (2), (3)
                        Rho_kl = table_data['Correlation'][id]
                
                    # According to 21.6 (1)
                    Sigma_Sigma_Rho_WS_k_WS_l_medium += Rho_kl * WS[k] * WS[l]
                    # According to 21.6 (2)
                    Sigma_Sigma_Rho_WS_k_WS_l_high += min(Rho_kl * 1.25, 1) * WS[k] * WS[l]
                    # According to 21.6 (3)
                    Sigma_Sigma_Rho_WS_k_WS_l_low += max(2 * Rho_kl - 1, 0.75 * Rho_kl) * WS[k] * WS[l]
        
        # Delta Risk Table Update according tol Medium Correlation, Hight Correlation, Low Correlation
        K_b[id,0] = np.sqrt(max(0, Sigma_WS_k_square + Sigma_Sigma_Rho_WS_k_WS_l_medium))
        K_b[id,1] = np.sqrt(max(0, Sigma_WS_k_square + Sigma_Sigma_Rho_WS_k_WS_l_high))
        K_b[id,2] = np.sqrt(max(0, Sigma_WS_k_square + Sigma_Sigma_Rho_WS_k_WS_l_low))

# Display the Result
print('Final Result:')
print('K_b in Midium Correlation, High Correlation, Low Correlation')
print('------------------------------------------------------------')
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.2f}".format(x)})
print(K_b)

Final Result:
K_b in Midium Correlation, High Correlation, Low Correlation
------------------------------------------------------------
[[109.34 112.45 106.14]
 [119.28 122.67 115.79]
 [89.46 92.01 86.84]
 [109.34 112.45 106.14]
 [64.06 66.68 61.34]
 [74.74 77.79 71.56]
 [85.42 88.91 81.78]
 [106.77 111.13 102.23]
 [130.89 133.00 128.73]
 [97.47 99.88 95.00]
 [211.40 211.40 211.40]
 [42.17 45.30 38.79]
 [70.29 75.50 64.65]]


### 5. Data Frame으로 저장

In [13]:
index = table_data['Bucket number']
column_name = ['midium correlation', 'high correlation', 'low correlation']
Delta_Risk_Table = pd.DataFrame(K_b, index= index, columns = column_name).round(2)
Delta_Risk_Table

,midium correlation,high correlation,low correlation
Bucket number,,,
1,109.34,112.45,106.14
2,119.28,122.67,115.79
3,89.46,92.01,86.84
4,109.34,112.45,106.14
5,64.06,66.68,61.34
6,74.74,77.79,71.56
7,85.42,88.91,81.78
8,106.77,111.13,102.23
9,130.89,133.00,128.73


In [14]:
Delta_Risk_Table.head()

,midium correlation,high correlation,low correlation
Bucket number,,,
1,109.34,112.45,106.14
2,119.28,122.67,115.79
3,89.46,92.01,86.84
4,109.34,112.45,106.14
5,64.06,66.68,61.34


In [15]:
Delta_Risk_Table.tail()

,midium correlation,high correlation,low correlation
Bucket number,,,
9,130.89,133.00,128.73
10,97.47,99.88,95.00
11,211.40,211.40,211.40
12,42.17,45.30,38.79
13,70.29,75.50,64.65


### 6. Final Code

In [16]:
import pandas as pd
import numpy as np
import copy

equity_file_name = 'equity.xlsx'
table_file_name = 'tables.xlsx'

equity_data = pd.read_excel(equity_file_name)
table_data  = pd.read_excel(table_file_name, sheet_name= 'table10', header=1)


# Define s_k: 0 <= k < num_equity_data
num_equity_data = np.size(equity_data,0)
Sensitivity = np.zeros(shape=(num_equity_data,1))
Sensitivity = equity_data['net sensitivity'].to_numpy()

# Define RW_k: 0<= k <num_equity_data
Risk_Weight = np.zeros(shape=(num_equity_data,1))
iter = 0
for buck_num,type in zip(equity_data['bucket'], equity_data['risk factor type']):
    if type == 'equity spot price':
        Risk_Weight[iter]= table_data['Risk weight for equity spot price'].to_numpy()[buck_num-1]
    else:
        Risk_Weight[iter]= table_data['Risk weight for equity repo rate'].to_numpy()[buck_num-1]
    iter = iter + 1
Risk_Weight= np.squeeze(Risk_Weight, axis=1)

# Define WS_k: 0<= k <num_equity_data
Weight_Sensitivity = Risk_Weight * Sensitivity


# Calculating Rho and Delta Risk Table Update according tol Medium Correlation, Hight Correlation, Low Correlation
Bucket = list(range(13))
K_b = np.zeros((13,3))

for id in Bucket:
    # Finding equity in the same bucket
    With_in_Bucket = equity_data[equity_data['bucket']==id+1]
    With_in_Bucket_index = np.where(equity_data['bucket'].to_numpy() == id+1)
    
    # Loading Weighted Sensitivity, Risk Factor Type, and Issuer Name in the same bucket
    # for calculating correlation, rho_kl
    Risk_Factor_Type = With_in_Bucket['risk factor type'].to_numpy()
    Issuer_Name = With_in_Bucket['issuer name'].to_numpy()
    WS = Weight_Sensitivity[With_in_Bucket_index]
    
    # the number of equity in the Bucket id+1
    len = np.size(With_in_Bucket,0)
    
    # Other sector bucket (Only for Bucket 11)
    if id == 10:
        for k in range(len):
            # According to 21.79 (1)
            K_b[id] += np.abs(WS[k])
    
    # within a bucket (Bucket 1,2,3,4,5,6,7,8,9,10,12,13)
    else:    
        Sigma_WS_k_square = 0
        Sigma_Sigma_Rho_WS_k_WS_l_medium = 0
        Sigma_Sigma_Rho_WS_k_WS_l_high = 0
        Sigma_Sigma_Rho_WS_k_WS_l_low = 0

        # First Summation (Sigma for k)
        for k in range(len):     
            # Second Summation(Sigma for all l != k)
            for l in range(len):
                if k == l:
                    Sigma_WS_k_square += WS[k] * WS[k]
                else:
                    if Risk_Factor_Type[k] != Risk_Factor_Type[l]:
                        if Issuer_Name[k] == Issuer_Name[l]:
                            # According to 21.78 (1)
                            Rho_kl = 0.9990             
                        else:
                            # 21.78 (4)
                            Rho_kl = 0.9990 * table_data['Correlation'][id]
                    else:
                        #  According to 21.78 (2), (3)
                        Rho_kl = table_data['Correlation'][id]

                    # According to 21.6 (1)
                    Sigma_Sigma_Rho_WS_k_WS_l_medium += Rho_kl * WS[k] * WS[l]
                    # According to 21.6 (2)
                    Sigma_Sigma_Rho_WS_k_WS_l_high += min(Rho_kl * 1.25,1) * WS[k] * WS[l]
                    # According to 21.6 (3)
                    Sigma_Sigma_Rho_WS_k_WS_l_low += max(2 * Rho_kl - 1, 0.75 * Rho_kl) * WS[k] * WS[l]

        # Delta Risk Table Update according tol Medium Correlation, Hight Correlation, Low Correlation
        K_b[id,0] = np.sqrt(max(0, Sigma_WS_k_square + Sigma_Sigma_Rho_WS_k_WS_l_medium))
        K_b[id,1] = np.sqrt(max(0, Sigma_WS_k_square + Sigma_Sigma_Rho_WS_k_WS_l_high))
        K_b[id,2] = np.sqrt(max(0, Sigma_WS_k_square + Sigma_Sigma_Rho_WS_k_WS_l_low))

index = table_data['Bucket number']
column_name = ['midium correlation', 'high correlation', 'low correlation']
Delta_Risk_Table = pd.DataFrame(K_b, index= index, columns = column_name).round(2)
Delta_Risk_Table

,midium correlation,high correlation,low correlation
Bucket number,,,
1,109.34,112.45,106.14
2,119.28,122.67,115.79
3,89.46,92.01,86.84
4,109.34,112.45,106.14
5,64.06,66.68,61.34
6,74.74,77.79,71.56
7,85.42,88.91,81.78
8,106.77,111.13,102.23
9,130.89,133.00,128.73
